# MEDPC Data Tutorial

**About:** The purpose of this tutorial is to tidy and plot data. 

**Contact:**
* Dexter Kim: dexterkim2000@gmail.com
* Brett Hathaway: bretthathaway@psych.ubc.ca

**Requirements**
* The data must be an excel file from MEDPC2XL (trial by trial data) 
* The data, rgt_functions.py file, and this notebook must be in your current working directory

**Note: This tutorial is split into multiple sections**
* Section 1: Setting Variables (objects) 
* Section 2: Loading Data into Python 
* Section 3: Acquisition analysis and plotting
* Section 4: Latin Square analysis and plotting
* Section 5: Miscellaneous

**Please run the following cell!**

In [1]:
%load_ext autoreload
%autoreload 2

In [15]:
#MEDPC rat gambling task functions imports, will print "I am being executed!" if functional
import rgt_functions as rgt

#main imports 
import os
import pandas as pd
import numpy as np

# plotting imports 
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

# stats imports 
import scipy.stats as stats

#the following line prevents pandas from giving unecessary errors 
pd.options.mode.chained_assignment = None

library(tidyverse)

NameError: name 'library' is not defined

# 1) Setting Variables (objects) 

Set your variables! These will be used later in the code. Example arguments are left in for clarity.

###Brett will edit

In [3]:
#in file_names (List[str]), add the file names you wish to read into Python 
# file_names = ['BH07_raw_free_S29-30.xlsx'] ###

#in group_names (List[str]), add the names of the control and experimental group, respectively 
group_names = ['Tg negative','Tg positive'] 

title = 'Nigrostriatal activation during acquisition' #title for figures, describing the experiment
startsess = 29 #first session in this dataset
endsess = 30 #last session in this dataset

group_names_2 = {0: 'tg negative',
              1: 'tg positive'} ###group_names is used twice (as a list, and as a dict)

#the following two lines of code assign the rat subject numbers to the experimental and control group lists
exp_group = [1, 2, 7, 8, 11, 12, 16, 19, 20, 21, 22, 25, 26, 29, 32] #Tg positive

control_group = [3, 4, 5, 6, 9, 13, 14, 15, 17, 18, 23, 24, 27, 28, 30, 31] #Tg negative

###for Latin Square analysis
file_names = ['BH06_raw_round1-infusions.xlsx', 'BH06_raw_round1-makeup.xlsx']

ls_group_names = {0:'lOFC',
               1:'PrL'} 

lOFC = [1,2,3,4,5,8,9,11,12,13,14,15,16]

PrL = [6,7] ##have to change this back with the new data

ls_groups = [lOFC, PrL]

###suggestion, make a data cell for section 3 and 4
###solution: move the variables to section 3 and 4 

# 2) Load data into Python
* `load_data()` takes in one argument: file_names 
* `load_data()` outputs a table similar to the excel sheet(s) you loaded in. (in the order established in `file_names`) 
* note: `df` means dataframe, and is an object that will store your dataframe (table containing your data) 
* passing `reset_sessions = True` ###makes the sessions start from 1 again (you may want to do this for baseline analysis)
* `load_multiple_data()` ###loads in multiple cohorts (with the same subject numbers) and assigns them unique subject numbers (ex. subject 1 of cohort 1 --> subject 101) 

In [4]:
df = rgt.load_data(file_names)

#load_data won't output a dataframe itself. Use the following function to view the top of your dataframe. Note: it should look the exact same as your first excel file. 

df.head()

,MSN,StartDate,StartTime,Subject,Group,Box,Experiment,Comment,Session,Trial,...,Pun_Persev_H5,Pun_HeadEntry,Pun_Dur,Premature_Resp,Premature_Hole,Rew_Persev_H1,Rew_Persev_H2,Rew_Persev_H3,Rew_Persev_H4,Rew_Persev_H5
0,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,1.1,...,0,0,0,1,5,0,0,0,0,0
1,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,1.1,...,0,0,0,1,4,0,0,0,0,0
2,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,1.0,...,2,1,30,0,0,0,0,0,0,0
3,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,2.1,...,0,0,0,1,5,0,0,0,0,0
4,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,2.1,...,0,0,0,1,5,0,0,0,0,0


# 3A) Acquisition Analysis Section (Analysis by Session)

**Check your session data**
* `check_sessions` gives us a summary for each rat (subject) including session numbers, session dates and # trials for each session.
* This allows us to see if there are any missing/incorrect session numbers, and if MED-PC exported all of the desired data into the Excel file you loaded in (`file_names`). 
* `edit_sessions()` can change session numbers (not included in the tutorial) 

In [5]:
rgt.check_sessions(df)

Subject  StartDate            Session
1        02/13/20             55          50.1
         02/17/20             57          79.1
         02/20/20             59          40.0
         02/24/20             61         111.0
2        02/13/20             55          94.0
         02/17/20             57          70.0
         02/20/20             59          74.0
         02/24/20             61          77.0
3        02/13/20             55          45.0
         02/17/20             57          50.1
         02/20/20             59          40.0
         02/24/20             61          38.1
4        02/13/20             55          82.0
         02/17/20             57          79.0
         02/20/20             59          86.0
         02/24/20             61          95.0
5        2003-01-20 00:00:00  65          87.0
         2003-05-20 00:00:00  69         128.0
         02/13/20             55          79.1
         02/27/20             63          79.0
6        02/13/20     

**To drop/remove data from certain session(s)**
* replace `session_num` with the session number data you want to remove
* for example, to remove all data from session 28 and 29, I would write: `rgt.drop_sessions(df, [28, 29])`
* **make sure to remove the correct session(s)**, if you remove the wrong session and want to put the data back, you'll have to restart the Kernel and restart from `load_data`

###do you want drop_sessions to be part of the tutorial? Or edit_sessions()? ###solution: include 

In [6]:
# rgt.drop_sessions(df, session_num) #session_num is a list of integers
#needs to be assigned to new object
df2 = rgt.drop_sessions(df, [28])

In [7]:
### this is a cell showing that it works --> 30 becomes 29, and 31 becomes 30, for subjects 17 to 32

# rgt.edit_sessions(df2, [30, 31], [29, 30], subs = "all")

**Check that you dropped the session desired (in this example, we dropped data from session 28)**

In [8]:
rgt.check_sessions(df) #df2

Subject  StartDate            Session
1        02/13/20             55          50.1
         02/17/20             57          79.1
         02/20/20             59          40.0
         02/24/20             61         111.0
2        02/13/20             55          94.0
         02/17/20             57          70.0
         02/20/20             59          74.0
         02/24/20             61          77.0
3        02/13/20             55          45.0
         02/17/20             57          50.1
         02/20/20             59          40.0
         02/24/20             61          38.1
4        02/13/20             55          82.0
         02/17/20             57          79.0
         02/20/20             59          86.0
         02/24/20             61          95.0
5        2003-01-20 00:00:00  65          87.0
         2003-05-20 00:00:00  69         128.0
         02/13/20             55          79.1
         02/27/20             63          79.0
6        02/13/20     

**Run the following cell to acquire a summary of your data.**

The rows represent subjects (rats 1 to n)

The columns are explained below:
* `##P#` represents the percent choice of each option. For example, `29P1` represents the percentage of times P1 was selected during the 29th session. 
* `risk##` represents the risk score for each session: (P1 + P2) - (P3 + P4) 
* `collect_lat##` represents the mean collect latency for each session
* `choice_lat##` represents the mean choice latency for each session 
* `trial##` represents the number of trials (not including premature responses or omissions) for each session
* `prem##` represents the number of premature responses for each session

In [9]:
df_sum = rgt.get_summary_data(df)
df_sum #prints the dataset 

,55P1,55P2,55P3,55P4,57P1,57P2,57P3,57P4,59P1,59P2,...,trial78,prem55,prem57,prem59,prem61,prem63,prem65,prem69,prem76,prem78
1,0,34,64,2,0,57.1429,35.0649,7.79221,5,40,...,NaN,54.954955,32.478632,70.588235,14.615385,NaN,NaN,NaN,NaN,NaN
2,15.0538,64.5161,2.15054,18.2796,8.57143,45.7143,7.14286,38.5714,12.3288,61.6438,...,NaN,17.543860,31.372549,28.846154,23.000000,NaN,NaN,NaN,NaN,NaN
3,2.22222,20,66.6667,11.1111,0,18,78,4,10,15,...,NaN,64.000000,57.264957,56.989247,70.400000,NaN,NaN,NaN,NaN,NaN
4,1.21951,65.8537,0,32.9268,0,61.039,3.8961,35.0649,1.23457,67.9012,...,NaN,21.904762,15.957447,24.561404,16.666667,NaN,NaN,NaN,NaN,NaN
5,5.12821,35.8974,55.1282,3.84615,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,33.898305,NaN,NaN,NaN,41.044776,21.621622,19.496855,NaN,NaN
6,5.78512,70.2479,1.65289,22.314,1.90476,69.5238,0,28.5714,2.06186,72.1649,...,NaN,1.626016,10.256410,15.517241,13.888889,NaN,NaN,NaN,NaN,NaN
7,7.35294,58.8235,23.5294,10.2941,5.33333,62.6667,16,16,8.77193,52.6316,...,NaN,50.000000,42.307692,52.892562,40.740741,NaN,NaN,NaN,NaN,NaN
8,6.89655,31.0345,5.17241,56.8966,6.06061,45.4545,4.54545,43.9394,4,44,...,NaN,42.574257,41.071429,18.478261,47.169811,NaN,NaN,NaN,NaN,NaN
9,0,1.5625,98.4375,0,0,14.9254,82.0896,2.98507,NaN,NaN,...,48.0,26.436782,24.175824,NaN,NaN,NaN,NaN,NaN,46.078431,54.716981
11,3.63636,0,94.5455,1.81818,1.63934,4.91803,93.4426,0,1.72414,3.44828,...,NaN,29.487179,11.594203,31.764706,35.294118,NaN,NaN,NaN,NaN,NaN


**Get the risk status of the rats using the following code**
* Note: 
    * `risk_status == 1` indicates a positive risk score (optimal) 
    * `risk_status == 2` indicates a negative risk score (risky)
    * `mean_risk` is the mean risk score averaged across the sessions between `startsess` and `endsess` for a given subject
        * you can change `startsess` and `endsess` by passing the session numbers instead. For example, `rgt.get_risk_status(df_sum, 28, 30)`
    * `print(risky, optimal)` prints out 2 list of rat subjects: the risky rats, and the optimal rats 

In [10]:
df_summary, risky, optimal = rgt.get_risk_status(df_sum, startsess, endsess)

print(df_summary[['mean_risk','risk_status']]) #printed 2 of many columns in df_summary ###this could be removed? ###solution: keep it
print(risky, optimal) #prints 2 lists: the subject numbers of the risky rats, and the subject numbers of the optimal rats

KeyError: 'risk29'

**Export your data to an Excel file!** 
* Note: `'tg_status'` is the column name that specifies the control vs. experimental group
* Note2: `'BH07_free_S29-30.xlsx'` specifies the name of the **new** Excel file 

###may want to change file_name to new_file_name for clarity ###change it 

In [ ]:
rgt.export_to_excel(df_summary, [control_group, exp_group], column_name = 'tg_status', file_name = 'BH07_free_S29-30.xlsx')

##Brett will edit control_group

**Summarize your data by experimental/control set**
* if you only want to view certain columns, specify them in mean_scores 
    * For example, `mean_scores[['risk29', 'risk30']]` will create a table with only those 2 columns
    * Each value is the mean for that column (ex. `29P1`) within the set (`tg negative` or `tg positive`) ###is this correct?
    
###in addition, it's strange that we have control_group and exp_group as the objects for 'tg_negative' and 'tg_positive', but I think you already noticed this. 

In [ ]:
mean_scores, stderror = rgt.get_group_means_sem(df_summary, [control_group, exp_group], group_names_2) ###[control_group, exp_group] = groups 
mean_scores #all mean scores
# mean_scores[['risk29', 'risk30']] #specify columns

# 3B) Acquisition Analysis (Plotting Section)

**Graph of the table above**
* `variable` specifies the variable you want to plot. 
    * For example, if I want to plot `choice_lat` over sessions for the experimental and control group, I would replace `variable` with `'choice_lat'`
* `startsess` and `endsess` can also be replaced with the range of session numbers you'd like to plot 
    * For example, if I want to plot `choice_lat` over sessions 29 to 31, I would replace `startsess` and `endsess` with `29` and `31` respectively

###this could be improved in description - 
###why does typing 'risk' work? ###explain what they can type into variables 

In [ ]:
rgt.rgt_plot('risk', startsess, endsess, group_names_2, title, mean_scores, stderror, var_title = 'risk score') ##group_names_2 has to be a dict not a list 

**Transforms the above data from a line plot, to a bar plot** 
* Must use the same arguments ##hard for me to tell why this would occur

In [ ]:
rgt.rgt_bar_plot('risk', startsess, endsess, group_names, title, mean_scores, stderror, var_title = None)

**Bar plot of P1-P4 choices**
* The following bar plot plots the mean P1-P4 choices for the tg negative and tg positive groups 

In [ ]:
rgt.choice_bar_plot(startsess, endsess, mean_scores, stderror)

# 4A) Latin Square Analysis (Analysis by Group) 

* Please note! This is the same workflow as the acquisition analysis 

**Check your group data**
* This will show you the number of trials performed by each subject-group pairing. 

In [11]:
rgt.check_groups(df)

Subject  Group
1        1         50.1
         2         79.1
         3         40.0
         4        111.0
2        1         77.0
         2         74.0
         3         70.0
         4         94.0
3        1         50.1
         2         38.1
         3         45.0
         4         40.0
4        1         86.0
         2         82.0
         3         95.0
         4         79.0
5        0         87.0
         1         79.1
         2         79.0
         4        128.0
6        1        125.1
         2         98.0
         3        105.0
         4        121.0
7        1         76.1
         2         80.1
         3         68.0
         4         57.1
8        1         75.0
         2         58.0
         3         57.1
         4         66.0
9        1         65.1
         2         69.0
         3         55.0
         4         48.0
11       1         56.1
         2         58.0
         3         61.0
         4         55.0
12       1         63.0
 

**Edit your group data**
* For example, if I want to change 0 to 1 for all subjects, I would write `rgt.edit_groups(df, orig_sess = [0], new_sess = [1], subs = "all")`
    * If I want to do the same thing but only for subject 2 and 3, change `subs = "all"` to `subs = [2,3]`
* For example, if I want to remove the data for subjects 5, 9 and 12, I would write `rgt.drop_subjects(df, subs = [5, 9, 12])`

In [12]:
rgt.edit_groups(df, orig_group = [0], new_group = [3], subs = [5])

# rgt.drop_subjects(df, subs = [5, 9, 12])

,index,MSN,StartDate,StartTime,Subject,Group,Box,Experiment,Comment,Session,...,Pun_HeadEntry,Pun_Dur,Premature_Resp,Premature_Hole,Rew_Persev_H1,Rew_Persev_H2,Rew_Persev_H3,Rew_Persev_H4,Rew_Persev_H5,option
0,0,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,...,0,0,1,5,0,0,0,0,0,0
1,1,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,...,0,0,1,4,0,0,0,0,0,0
2,2,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,...,1,30,0,0,0,0,0,0,0,3
3,3,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,...,0,0,1,5,0,0,0,0,0,0
4,4,rGT_A-cue,02/13/20,15:35:41,9,1,1,0,NaN,55,...,0,0,1,5,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,6415,rGT_A-cue,03/19/20,13:48:44,9,4,1,0,NaN,78,...,0,0,0,0,0,0,0,0,0,3
6416,6416,rGT_A-cue,03/19/20,13:48:44,9,4,1,0,NaN,78,...,0,0,1,5,0,0,0,0,0,0
6417,6417,rGT_A-cue,03/19/20,13:48:44,9,4,1,0,NaN,78,...,0,0,1,5,0,0,0,0,0,0
6418,6418,rGT_A-cue,03/19/20,13:48:44,9,4,1,0,NaN,78,...,0,0,0,0,0,0,0,0,0,3


**Check that you edited the group desired**

In [13]:
rgt.check_groups(df)

Subject  Group
1        1         50.1
         2         79.1
         3         40.0
         4        111.0
2        1         77.0
         2         74.0
         3         70.0
         4         94.0
3        1         50.1
         2         38.1
         3         45.0
         4         40.0
4        1         86.0
         2         82.0
         3         95.0
         4         79.0
5        1         79.1
         2         79.0
         3         87.0
         4        128.0
6        1        125.1
         2         98.0
         3        105.0
         4        121.0
7        1         76.1
         2         80.1
         3         68.0
         4         57.1
8        1         75.0
         2         58.0
         3         57.1
         4         66.0
9        1         65.1
         2         69.0
         3         55.0
         4         48.0
11       1         56.1
         2         58.0
         3         61.0
         4         55.0
12       1         63.0
 

**Run the following cell to acquire a summary of your data.**

The rows represent subjects (rats 1 to n)

The columns are explained below:
* `##P#` represents the percent choice of each option. For example `1P1` represents the percent choice of P1 
* `risk##` represents the risk score for each group: (P1 + P2) - (P3 + P4) 
* `collect_lat##` represents the mean collect latency for each group
* `choice_lat##` represents the mean choice latency for each group
* `trial##` represents the number of trials (not including premature responses or omissions) for each group
* `prem##` represents the number of premature responses for each group

In [14]:
df1 = rgt.get_summary_data(df, mode = 'Group')
df1

,1P1,1P2,1P3,1P4,2P1,2P2,2P3,2P4,3P1,3P2,...,omit3,omit4,trial1,trial2,trial3,trial4,prem1,prem2,prem3,prem4
1,0,34,64,2,0,57.1429,35.0649,7.79221,5,40,...,0,2,50.1,79.1,40.0,111.0,54.954955,32.478632,70.588235,14.615385
2,3.8961,59.7403,7.79221,28.5714,12.3288,61.6438,6.84932,19.1781,8.57143,45.7143,...,0,1,77.0,74.0,70.0,94.0,23.000000,28.846154,31.372549,17.543860
3,0,18,78,4,0,13.8889,80.5556,5.55556,2.22222,20,...,0,0,50.1,38.1,45.0,40.0,57.264957,70.400000,64.000000,56.989247
4,1.23457,67.9012,0,30.8642,1.21951,65.8537,0,32.9268,0,77.6596,...,1,2,86.0,82.0,95.0,79.0,24.561404,21.904762,16.666667,15.957447
5,5.12821,35.8974,55.1282,3.84615,3.84615,61.5385,28.2051,6.41026,17.4419,40.6977,...,1,0,79.1,79.0,87.0,128.0,33.898305,41.044776,21.621622,19.496855
6,1.62602,85.3659,0,13.0081,2.06186,72.1649,0,25.7732,1.90476,69.5238,...,0,0,125.1,98.0,105.0,121.0,13.888889,15.517241,10.256410,1.626016
7,5.33333,62.6667,16,16,21.25,58.75,2.5,17.5,7.35294,58.8235,...,0,0,76.1,80.1,68.0,57.1,42.307692,40.740741,50.000000,52.892562
8,4,44,0,52,6.89655,31.0345,5.17241,56.8966,0,42.8571,...,0,0,75.0,58.0,57.1,66.0,18.478261,42.574257,47.169811,41.071429
9,0,1.5625,98.4375,0,0,14.9254,82.0896,2.98507,1.85185,11.1111,...,1,2,65.1,69.0,55.0,48.0,26.436782,24.175824,46.078431,54.716981
11,3.63636,1.81818,94.5455,0,1.72414,3.44828,93.1034,1.72414,1.63934,4.91803,...,0,0,56.1,58.0,61.0,55.0,35.294118,31.764706,11.594203,29.487179


###Impute your missing data 
* For example, if you have missing data for subject 12, session 2, you can impute (take the mean of the session before and after). 
    * Code: `rgt.impute_missing_data(df, session = 2, subject = 12, choice = 'all', vars = 'all')`

In [ ]:
df_group_summary = rgt.impute_missing_data(df1, session = 2, subject = 12, choice = 'all', vars = 'all')
df_group_summary

**Summarize your data by experimental/control set**
* If you only want to view certain columns, specify them in group_means 
    * For example, `group_means[['omit3', 'omit4']]` will create a table with only those 2 columns ###change
    * Each value is the mean for that column (ex. `omit3`) within the set (`lOFC` or `PrL`) ###is this correct?
    
###this doesn't work - something about stuff being not supported... didn't have time to rewrite it. I really tried...
###solutionY: PrL and lOFC cannot contain subject labels that aren't present in the data --> KeyError
###solutionX: PrL and lOFC must contain at least 2 subjects. 
###solution: check that you have X and Y 

In [ ]:
group_means, sem = rgt.get_group_means_sem(df1, ls_groups, ls_group_names) 
group_means
# group_means[['omit3', 'omit4']]

**Get risk status of the vehicle**

###This skipped a lot of steps included in the BH06 LS analysis ###have risky vs optimal graphs, and also have by lOFC/PrL 
###This is obviously not exported... but I can include it before export

In [ ]:
df1,risky,optimal = rgt.get_risk_status_vehicle(df1) 
df1

In [ ]:
#make lists for risky and optimal rats in each group
#magical function that does that (value for value)

#calculate means for risky and optimal by group


#group_means_risk, sem_risk = rgt.get_group_means_sem(df1, ls_groups_risk, ls_group_names_risk) 

**Export your data to an Excel file!** 
* Note: `'tg_status'` is the column name that specifies the control vs. experimental group
* Note2: `'BH07_free_S29-30.xlsx'` specifies the name of the **new** Excel file 

###may want to change file_name to new_file_name for clarity 
###does work!

In [ ]:
rgt.export_to_excel(df1,ls_groups,'brain_region','BH06_all-data2.xlsx')

# 4B) Plotting Section (by Groups) 

**Graph of the table above**
* `variable` specifies the variable you want to plot. 
* For example, if I want to plot `choice_lat` over sessions for the experimental and control group, I would replace `variable` with `'choice_lat'`

##this could be improved in description

In [ ]:
rgt.ls_bar_plot('lOFC',group_means,sem)

In [ ]:
rgt.ls_bar_plot('PrL',group_means,sem)

In [ ]:
rgt.rgt_plot('omit',1,4,ls_group_names,'5-HT2c Antagonist',group_means,sem,var_title = 'Premature response')

In [ ]:
#plots for risky vs optimal 
#use group names to specify choice plot for risky or optimal rats 

In [ ]:
#line plot with risky and optimal rats from both groups

# 5) Miscellaneous Section (more advanced code) 

**Change your working directory**

Instructions: 
* Check your current working directory by running line 2. 
* From your working directory, make a data folder (call it: data), and add your .xlsx file into that folder. 
* Change `('C:\\Users\\dexte\\hathaway_1\\data')` to your current working directory and add '\\data'
* For example, my current working directory is `'C:\\Users\\dexte\\hathaway_1'`, so I enter `'C:\\Users\\dexte\\hathaway_1\\data'` into the brackets (slashes will be different if you are not using windows). 
* This saves all data in your data folder, instead of your current working directory. 

##default: just have their data in their cwd (easier option)
##future: write a function that will save files in separate folder (for them)

In [ ]:
#checks current working directory
os.getcwd()

#changes working directory to whatever is included in brackets
os.chdir('C:\\Users\\dexte\\hathaway_1\\data') 